In [1]:
!pip install "spacy >= 3.0.6"
!pip install -U wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=97d09163c83d94b08e732559ffde98cc2a2860a4ebb384f6440b478231afcdbe
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: url

In [2]:
import os
from pathlib import Path
import wandb
import spacy

In [3]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
spacy_artifact = "wandb/spacy/spacy_demo:v3"
spacy_dir = Path("my_spacy_demo") 

In [12]:
with wandb.init(project="spacy_demo") as run:
  artifact = run.use_artifact(spacy_artifact)
  _ = artifact.download(spacy_dir)

wandb:   4 of 4 files downloaded.  


In [13]:
!python -m spacy train my_spacy_demo/configs/cnn.cfg \
  --paths.train my_spacy_demo/corpus/train.spacy \
  --paths.dev my_spacy_demo/corpus/dev.spacy \
  -o my_spacy_demo/training/cnn \
  --gpu-id 0

✔ Created output directory: my_spacy_demo/training/cnn
ℹ Saving to output directory: my_spacy_demo/training/cnn
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-02-07 14:55:40,454] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-02-07 14:55:40,906] [INFO] Pipeline: ['textcat']
INFO:spacy:Pipeline: ['textcat']
[2023-02-07 14:55:40,910] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2023-02-07 14:55:40,911] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2023-02-07 14:56:12,446] [INFO] Initialized pipeline components: ['textcat']
INFO:spacy:Initialized pipeline components: ['textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat']
ℹ Initial learn rate: 0.001
wandb: Currently logged in as: eshban9492. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb ver

In [15]:
texts = ["This is a fabulous idea, this made be so happy and excited, I want to jump for joy",
        "This movie was terrifying, I jumped out of my seat I was so scared, I never want to watch this again"]

In [16]:
nlp = spacy.load("my_spacy_demo/training/cnn/model-best")

In [17]:
category_scores = [doc.cats for doc in nlp.pipe(texts)]
category_scores[0]

{'admiration': 0.4571589529514313,
 'amusement': 0.08408202230930328,
 'anger': 0.0015716038178652525,
 'annoyance': 0.0034235757775604725,
 'approval': 0.014366304501891136,
 'caring': 0.00404041213914752,
 'confusion': 0.0019566311966627836,
 'curiosity': 0.009191148914396763,
 'desire': 0.08018191158771515,
 'disappointment': 0.021105658262968063,
 'disapproval': 0.0012093916302546859,
 'disgust': 0.012177985161542892,
 'embarrassment': 0.01231912337243557,
 'excitement': 0.21220272779464722,
 'fear': 0.043677832931280136,
 'gratitude': 0.013848154805600643,
 'grief': 0.0010895136510953307,
 'joy': 0.9681096076965332,
 'love': 0.001000059419311583,
 'nervousness': 0.005912921391427517,
 'optimism': 0.005568949971348047,
 'pride': 0.005461690481752157,
 'realization': 0.003335074055939913,
 'relief': 0.013613969087600708,
 'remorse': 0.0011887188302353024,
 'sadness': 0.016887683421373367,
 'surprise': 0.004507335834205151,
 'neutral': 0.0004995354684069753}

In [18]:
threshold = 0.5
for d in category_scores:
  print(dict((k, v) for k, v in d.items() if v >= threshold))

{'joy': 0.9681096076965332}
{'fear': 0.9173496961593628}
